In [1]:
import cv2
import numpy as np
from glob import glob
from matplotlib import pyplot as plt

In [2]:
def patch_gen(img_orig,patchSize,r):
#     print(len(img_orig.shape))
    if(len(img_orig.shape)==3):
        img = cv2.cvtColor(img_orig,cv2.COLOR_RGB2GRAY)
    else:
        img = img_orig
#     plt.imshow(img, cmap='gray')
#     print(img.shape[1])
    x = np.random.randint(r, img.shape[1]-r-patchSize)  
    y = np.random.randint(r, img.shape[0]-r-patchSize) 

    p1 = (x,y)
    p2 = (patchSize+x, y)
    p3 = (patchSize+x, patchSize+y)
    p4 = (x, patchSize+y)
    src = [p1, p2, p3, p4]
    
#     # Printing and plotting Source Points
#     print(src)
#     plt.figure()
#     src_image = img_orig.copy()
#     cv2.polylines(src_image, np.int64([src]), 1, (255,0,0),2)
#     plt.imshow(src_image)
    
    dst = []
    for pt in src:
        dst.append((pt[0]+np.random.randint(-r, r), pt[1]+np.random.randint(-r, r)))
    
#     #Printing and Plotting Destination Points
#     print(dst)
#     plt.figure()
#     dst_image = img_orig.copy()
#     cv2.polylines(dst_image, np.int64([dst]), 1, (255,0,0),2)
#     plt.imshow(dst_image)

    H = cv2.getPerspectiveTransform(np.float32(src), np.float32(dst))
    H_inv = np.linalg.inv(H)
    warpImg = cv2.warpPerspective(img, H_inv, (img.shape[1],img.shape[0]))
    
#     # Plotting Warped Image
#     tempImg = warpImg.copy()
#     cv2.polylines(tempImg, np.int32([src]), 1, (0),2)
#     plt.figure()
#     plt.imshow(tempImg, cmap='gray')
#     plt.show()
    
    patch1 = img[y:y + patchSize, x:x + patchSize]
    patch2 = warpImg[y:y + patchSize, x:x + patchSize]
    
    
#     plt.figure()
#     plt.imshow(patch1, cmap='gray')
#     plt.show()
#     plt.figure()
#     plt.imshow(patch2, cmap='gray')
#     plt.show()
#     images = np.dstack(img_orig)
    imgData = np.dstack((patch1, patch2))
    hData = np.subtract(np.array(dst), np.array(src))
    return imgData,src

In [4]:
# trainImg = glob("./Data/Train/*.jpg")
# valImg = glob("./Data/Val/*.jpg")
Basepath = "./Data/Train/"
# print("No. of Training Images = " + str(len(trainImg)))
# print("No. of Validation Images = " + str(len(valImg)))


Patch=[]
Corner = []
size=(640,480)
for i in range(5000): 
    trainImg = Basepath + str(i+1)+".jpg"
    img = plt.imread(trainImg)
    img = cv2.resize(img,size)
    patches,src = patch_gen(img,128,32)
    Patch.append(patches)
    Corner.append(src)

In [5]:
import pickle

training = {'Patches':Patch,'Corners':Corner}
pickle.dump(training, open("unsup.pkl", "wb"))